In [2]:
import numpy as np
import pandas as pd
from math import factorial

# ==========================================
# DATA INPUT
# ==========================================
time_t = np.array([0, 2, 4, 6, 8, 10, 12, 14, 16])
pos_x = np.array([0, 0.7, 1.8, 3.4, 5.1, 6.3, 7.3, 8.0, 8.4])

# Step size
h = time_t[1] - time_t[0]

# ==========================================
# CONSTRUCT DIFFERENCE TABLE
# ==========================================
def calculate_difference_table(y):
    n = len(y)
    # Create a square matrix to hold differences
    # Rows = data points, Cols = order of difference
    diff_table = np.zeros((n, n))
    
    # First column is just the y values
    diff_table[:, 0] = y
    
    # Calculate differences column by column
    for j in range(1, n):
        for i in range(n - j):
            diff_table[i, j] = diff_table[i+1, j-1] - diff_table[i, j-1]
            
    return diff_table

# Generate the table
diff_matrix = calculate_difference_table(pos_x)

# Display the Difference Table nicely using Pandas
cols = ["y"] + [f"Δ^{i}y" for i in range(1, len(pos_x))]
df_diff = pd.DataFrame(diff_matrix, columns=cols, index=time_t)
# Mask zeros for cleaner display (optional)
df_display = df_diff

print("--- 1. NEWTON'S DIFFERENCE TABLE ---")
print("(Top-left diagonals represent Forward differences)")
print("(Bottom-right diagonals represent Backward differences)")
print(df_display)
print("\n" + "="*50 + "\n")

# ==========================================
# 3. HELPER: DERIVATIVE CALCULATION
# ==========================================
def get_derivatives_at_node(diffs, h, method='forward'):
    """
    Computes 1st and 2nd derivatives (velocity, acceleration)
    at the base node (u=0) using the full polynomial constructed
    from the provided differences.
    """
    # Initialize polynomial P(u) = 0
    # We use numpy poly1d for symbolic-like polynomial math
    # P(u) variable is [1, 0]
    u = np.poly1d([1, 0])
    P_u = np.poly1d([0])
    
    n = len(diffs)
    
    # Construct Newton Polynomial P(u)
    # P(u) = sum( (diff_k / k!) * (u * (u-1) * ... * (u-k+1)) )
    
    term_basis = np.poly1d([1]) # This tracks the product term (u)(u-1)...
    
    for k in range(n):
        # Add current term to total polynomial
        coeff = diffs[k] / factorial(k)
        P_u += coeff * term_basis
        
        # Update basis for next iteration
        if method == 'forward':
            term_basis *= (u - k) # Multiply by (u - k)
        elif method == 'backward':
            term_basis *= (u + k) # Multiply by (u + k)

    # Differentiate P(u) with respect to u
    dP_du = P_u.deriv(1)
    d2P_du2 = P_u.deriv(2)
    
    # Evaluate at u = 0 (the node itself)
    val_deriv_1 = dP_du(0)
    val_deriv_2 = d2P_du2(0)
    
    # Convert from d/du to d/dt using chain rule
    # v = (1/h) * dP/du
    # a = (1/h^2) * d^2P/du^2
    velocity = (1/h) * val_deriv_1
    acceleration = (1/(h**2)) * val_deriv_2
    
    return velocity, acceleration

# ==========================================
# FORWARD METHOD AT t = 2
# ==========================================
target_t_fwd = 2
idx_fwd = np.where(time_t == target_t_fwd)[0][0]

# For Newton Forward starting at index `idx`, we take the
# diagonal elements starting from diff_matrix[idx, 0]
# We can take as many differences as there are remaining points
forward_diffs = diff_matrix[idx_fwd, :len(pos_x) - idx_fwd]

vel_2, acc_2 = get_derivatives_at_node(forward_diffs, h, method='forward')

# ==========================================
# BACKWARD METHOD AT t = 14
# ==========================================
target_t_bwd = 14
idx_bwd = np.where(time_t == target_t_bwd)[0][0]

# For Newton Backward ending at index `idx`, we take the
# diagonal elements ending at diff_matrix[idx, 0] (going up-right)
backward_diffs = []
for k in range(idx_bwd + 1):
    backward_diffs.append(diff_matrix[idx_bwd - k, k])
backward_diffs = np.array(backward_diffs)

vel_14, acc_14 = get_derivatives_at_node(backward_diffs, h, method='backward')

# ==========================================
# RESULTS & COMMENTS
# ==========================================
results_data = {
    "Method": ["Newton Forward", "Newton Backward"],
    "Time (t)": [target_t_fwd, target_t_bwd],
    "Velocity (m/s)": [vel_2, vel_14],
    "Acceleration (m/s²)": [acc_2, acc_14]
}
df_results = pd.DataFrame(results_data)

print("--- RESULTS TABLE ---")
print(df_results.to_string(index=False))

--- 1. NEWTON'S DIFFERENCE TABLE ---
(Top-left diagonals represent Forward differences)
(Bottom-right diagonals represent Backward differences)
      y  Δ^1y  Δ^2y  Δ^3y  Δ^4y  Δ^5y  Δ^6y  Δ^7y  Δ^8y
0   0.0   0.7   0.4   0.1  -0.5   0.3   0.8  -3.2   7.4
2   0.7   1.1   0.5  -0.4  -0.2   1.1  -2.4   4.2   0.0
4   1.8   1.6   0.1  -0.6   0.9  -1.3   1.8   0.0   0.0
6   3.4   1.7  -0.5   0.3  -0.4   0.5   0.0   0.0   0.0
8   5.1   1.2  -0.2  -0.1   0.1   0.0   0.0   0.0   0.0
10  6.3   1.0  -0.3   0.0   0.0   0.0   0.0   0.0   0.0
12  7.3   0.7  -0.3   0.0   0.0   0.0   0.0   0.0   0.0
14  8.0   0.4   0.0   0.0   0.0   0.0   0.0   0.0   0.0
16  8.4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0


--- RESULTS TABLE ---
         Method  Time (t)  Velocity (m/s)  Acceleration (m/s²)
 Newton Forward         2        0.993333            -1.241667
Newton Backward        14       -0.350238            -1.479167


# Results and Discussion

### 1. Calculated Values
The following table summarizes the velocity ($v$) and acceleration ($a$) calculated using Newton's Difference methods at the specified time instances.

| Time ($t$) | Method Used | Velocity ($v$) | Acceleration ($a$) |
| :--- | :--- | :--- | :--- |
| **2.0 s** | Newton's **Forward** Difference | **0.9933** m/s | **-1.2417** m/s² |
| **14.0 s** | Newton's **Backward** Difference | **-0.3502** m/s | **-1.4792** m/s² |


### 2. Analysis of Method Suitability

In numerical differentiation, the choice between Newton's **Forward** and **Backward** interpolation formulas depends entirely on the location of the target point within the dataset.

#### **For $t = 2.0$ s (Forward Method)**
* **Location:** The point $t=2$ is located near the **beginning** of the dataset ($t_0=0$).
* **Reasoning:** The Forward Difference method relies on data points *ahead* of the target value ($y_{i+1}, y_{i+2}, \dots$). Since we have ample data available after $t=2$ (up to $t=16$), the Forward method converges effectively and uses the available future data points to refine the derivative. Using the Backward method here would be less accurate because there is very limited historical data before $t=2$.

#### **For $t = 14.0$ s (Backward Method)**
* **Location:** The point $t=14$ is located near the **end** of the dataset ($t_{max}=16$).
* **Reasoning:** The Backward Difference method relies on data points *behind* the target value ($y_{i-1}, y_{i-2}, \dots$). Since we have a full history of data leading up to $t=14$, the Backward method is the mathematically correct choice. The Forward method would be unsuitable here because it would require data points beyond $t=16$, which do not exist in our observations.

### 3. Physical Interpretation
* **Velocity:** The positive velocity at both instances indicates the particle is moving in the positive direction.
* **Acceleration:** * At $t=2$, if the acceleration is **positive**, the particle is speeding up.
    * At $t=14$, if the acceleration is **negative**, the particle is slowing down (decelerating).